In [1]:
import setup_runs as sr
import os, os.path
import pandas as pd
import numpy as np
import pyDOE as pyd
import time
import sys

#export files?
export_ed_files_q = True



In [2]:

print("Starting build of components for experimental design.")


#number of lhs samples
n_lhs = sr.dict_init["n_lhs"]
#set baseline strategy
strat_baseline = 0#[x for x in sr.dict_strat.keys() if sr.dict_strat[x]["strategy_id"] == 0][0]

###   SOME ATTRIBUTE TABLES
df_attribute_design_id = pd.read_csv(sr.fp_csv_attribute_design)
#reduce
df_attribute_design_id = df_attribute_design_id[df_attribute_design_id["include"] == 1]
#get time series id
df_attribute_time_series_id = pd.read_csv(sr.fp_csv_attribute_time_series)


print("Check: export_ed_files_q = " + str(export_ed_files_q))
print("Check: n_lhs = " + str(n_lhs))

    


#################################
#    MULTI SECTOR COMPONENTS    #
#################################


###   GET THE PARAMETER TABLE

#read in uncertainty table for additional sectors
if not sr.excursion_q:
    parameter_table_additional_sectors = pd.read_csv(sr.fp_csv_parameter_ranges)
else:
    parameter_table_additional_sectors = pd.read_csv(sr.fp_csv_parameter_ranges_for_excursion)
#reduce
parameter_table_additional_sectors = parameter_table_additional_sectors[parameter_table_additional_sectors["type"].isin(["incertidumbre", "accion"])]
#add field
parameter_table_additional_sectors["variable_name_lower"] = [x.lower().replace(" ", "_") for x in list(parameter_table_additional_sectors["parameter"])]
#fill nas and set to integer
parameter_table_additional_sectors["parameter_constant_q"] = parameter_table_additional_sectors["parameter_constant_q"].fillna(0)
parameter_table_additional_sectors["parameter_constant_q"] = np.array(parameter_table_additional_sectors["parameter_constant_q"]).astype(int)
#initialize available groups
groups_norm = set([int(x) for x in parameter_table_additional_sectors["normalize_group"] if not np.isnan(x)])

# IDENTIFY PARAMETERS THAT DO NOT VARY
df_apn = parameter_table_additional_sectors[["variable_name_lower", "min_2050", "max_2050"]].copy().drop_duplicates().reset_index(drop = True)
#initialize the set
all_params_novary = set({})
#loop to build
for p in df_apn["variable_name_lower"].unique():
    df_apn_tmp = df_apn[df_apn["variable_name_lower"] == p]
    #check to see if it doesn't vary
    if len(df_apn_tmp) == 1:
        if float(df_apn_tmp["min_2050"].iloc[0]) == float(df_apn_tmp["max_2050"].iloc[0]):
            if float(df_apn_tmp["min_2050"].iloc[0]) == 1.0:
                all_params_novary = all_params_novary | set({p})
#all parameters that vary
all_params_vary = set(parameter_table_additional_sectors["variable_name_lower"]) - all_params_novary




###   NORMALIZATION GROUP IDS

group_id = parameter_table_additional_sectors[["type", "parameter", "normalize_group"]].drop_duplicates()
#build normalize group and lever group ids (for deltas)
norm_vec = []

if len(groups_norm) > 0:
    #starting point for new group
    ind_group = max(groups_norm) + 1
else:
    ind_group = 1
    
#loop over rows
for i in range(0, len(group_id)):
    #get current group
    group_cur = group_id["normalize_group"].iloc[i]
    #test for NaN
    if np.isnan(group_cur):
        norm_vec = norm_vec + [ind_group]
        #next iteration
        ind_group += 1
    else:
        norm_vec = norm_vec + [int(group_cur)]

#add to group_id data frame
group_id["norm_group_id"] = norm_vec
group_id = group_id.reset_index(drop = True)


###   LEVER GROUP IDS

#temporary data frame to build lever group id with
group_id_tmp = group_id[["type", "norm_group_id"]].drop_duplicates()
group_id_tmp = group_id_tmp[group_id_tmp["type"].isin(["Accion", "accion"])]
group_id_tmp["lever_group_id"] = range(1, len(group_id_tmp) + 1)
if "lever_group_id" not in group_id.columns:
    group_id = pd.merge(group_id, group_id_tmp[["norm_group_id", "lever_group_id"]], how = "left", left_on = ["norm_group_id"], right_on = ["norm_group_id"])
#reduce
group_id = group_id[["parameter", "type", "norm_group_id", "lever_group_id"]]
#replace nas
group_id = group_id.fillna({"lever_group_id": -1})
#integer
group_id["lever_group_id"] = [int(x) for x in list(group_id["lever_group_id"])]
#merge in
group_id  = pd.merge(group_id, parameter_table_additional_sectors[["parameter", "variable_name_lower"]], how = "left", left_on = ["parameter"], right_on = ["parameter"])
group_id = group_id.drop_duplicates()
group_id = group_id[["parameter", "type", "variable_name_lower", "norm_group_id", "lever_group_id"]]



###   MERGE BACK IN TO PTAS AND CLEAN NAMES/DATA

#set merge fields and merge
fields_merge = list(set(parameter_table_additional_sectors.columns).intersection(set(group_id.columns)))
parameter_table_additional_sectors = pd.merge(parameter_table_additional_sectors, group_id, how = "left", left_on = fields_merge, right_on = fields_merge)
#fields to extract
fields_ext = [x for x in parameter_table_additional_sectors if (x not in ["normalize_group"])]
#clean the data frame
parameter_table_additional_sectors = parameter_table_additional_sectors[fields_ext]
#fields that are allowed to be all na
fields_allow_na = ["trajgroup_no_vary_q", "parameter_constant_q", "normalize_group"]
for field in fields_allow_na:
    if field in parameter_table_additional_sectors.columns:
        parameter_table_additional_sectors[field] = np.array(parameter_table_additional_sectors[field].fillna(0)).astype(int)
#drop nas
parameter_table_additional_sectors = parameter_table_additional_sectors.dropna(axis = 1, how = "all")

#dictionary to rename
dict_ptas_rename = dict([[x, x.lower().replace(" ", "_")] for x in parameter_table_additional_sectors.columns])
#set type
parameter_table_additional_sectors = parameter_table_additional_sectors.rename(columns = dict_ptas_rename)

#parameters to index
fields_add_sec_all_vals = ["strategy_id", "type", "parameter", "sector", "norm_group_id", "lever_group_id"]
#get parameter years that are defined
param_years_add_sec = [float(x) for x in parameter_table_additional_sectors.columns if x.replace(".", "").isnumeric()]
param_years_add_sec = [int(x) for x in param_years_add_sec if (int(x) == x)]

#initialize for all parameters
all_vals_add_sec = {
    "param_years": param_years_add_sec,
    "future_id": list(range(1, n_lhs + 1)),
    "design_id": list(df_attribute_design_id["design_id"]),
    "time_series_id": list(set(df_attribute_time_series_id["time_series_id"]) & set(parameter_table_additional_sectors["time_series_id"]))
}
#sort some
all_vals_add_sec["time_series_id"].sort()

#loop
for field in fields_add_sec_all_vals:
    #set the field name
    str_field = field.lower().replace(" ", "_")
    #
    if field in ["lever_group_id", "norm_group_id"]:
        set_field = set([x for x in parameter_table_additional_sectors[field] if x > 0])
    else:
        set_field = set(parameter_table_additional_sectors[field])
    #update the dictionary
    all_vals_add_sec.update({str_field: set_field})

    
    
    
###################################
#    GENERATE ATTRIBUTE TABLES    #
###################################

###   RUN ID
df_attribute_run_id_0 = [(x, y) for x in sr.df_strat["strategy_id"] for y in ([0] + all_vals_add_sec["future_id"])]
df_attribute_run_id = pd.DataFrame(df_attribute_run_id_0)
df_attribute_run_id = df_attribute_run_id.rename(columns = {0: "strategy_id", 1: "future_id"})
df_attribute_run_id["run_id"] = range(0, len(df_attribute_run_id))
df_attribute_run_id = df_attribute_run_id[["run_id", "strategy_id", "future_id"]]

if export_ed_files_q:
    #note/export
    print("Exporting run_id attribute to " + sr.fp_csv_attribute_runs)
    df_attribute_run_id.to_csv(sr.fp_csv_attribute_runs, index = None)

###   MASTER ID
df_attribute_master_id = [[x] + [y] + list(z) for x in all_vals_add_sec["design_id"] for y in all_vals_add_sec["time_series_id"] for z in df_attribute_run_id_0]
df_attribute_master_id = pd.DataFrame(df_attribute_master_id)
df_attribute_master_id = df_attribute_master_id.rename(columns = {0: "design_id", 1: "time_series_id", 2: "strategy_id", 3: "future_id"})
fields_df_attribute_master_id = list(df_attribute_master_id.columns)
df_attribute_master_id["master_id"] = list(range(0, len(df_attribute_master_id)))
df_attribute_master_id = df_attribute_master_id[["master_id"] + fields_df_attribute_master_id]
df_attribute_master_id = pd.merge(df_attribute_master_id, df_attribute_run_id, how = "left", on = ["strategy_id", "future_id"])
#fields to order by
fields_ord_dfm = ["master_id", "design_id", "time_series_id", "run_id", "strategy_id", "future_id"]
#reorder
df_attribute_master_id = df_attribute_master_id[fields_ord_dfm].sort_values(by = fields_ord_dfm)

if export_ed_files_q and not sr.tornado_q:
    #note/export
    print("Exporting master_id attribute to " + sr.fp_csv_attribute_master)
    #export
    df_attribute_master_id.to_csv(sr.fp_csv_attribute_master, index = None, encoding = "UTF-8")
    #export for gams
    df_attribute_master_id_gams = df_attribute_master_id[["master_id"]].copy().rename(columns = {"master_id": "Escenarios"})
    df_attribute_master_id_gams.to_csv(sr.fp_csv_gams_data_set_escenarios, index = None, encoding = "UTF-8")
    del df_attribute_master_id_gams


Starting build of components for experimental design.
Check: export_ed_files_q = True
Check: n_lhs = 1000
Exporting run_id attribute to /Users/jsyme/Documents/Projects/FY20/SWCHE102-1000/git/MultiSector_LTS_Chile/experimental_design/attribute_runs.csv


In [5]:
print("Building tornado design...\n")

####################################################
###                                              ###
###    BUILD TORNADO PLOT EXPERIMENTAL DESIGN    ###
###                                              ###
####################################################

#id fields
fields_id_ptas_internal = ["norm_group_id", "lever_group_id"]
fields_id_ptas = [x for x in parameter_table_additional_sectors.columns if (x[-3:] == "_id") and (x not in fields_id_ptas_internal)]
tuples_id_ptas = parameter_table_additional_sectors[fields_id_ptas].drop_duplicates()
tuples_id_ptas = [tuple(x) for x in np.array(tuples_id_ptas)]

field_param = "variable_name_lower"
fields_param_years_add_sec = [str(x) for x in param_years_add_sec]

#mix vector
y_0 = int(sr.dict_init["add_sec_variation_start_year"]) - 1
y_1 = max(param_years_add_sec)
y_base = min(param_years_add_sec)
vec_ramp_unc = sr.build_linear_mix_vec((y_0, y_1), (y_base, y_1))

#initialize loop for master id
m_id = 0
f_id = 0
#initialize attribute master id out
df_master_out = []

init_ed_q = True
init_fut_q = True

df_out = []

Building tornado design...



In [10]:
ti = tuples_id_ptas[0]

In [11]:
#copy the dataframe
params_tmp = parameter_table_additional_sectors.copy()
#reduce the dataframe t the approporiate subset
for j in range(len(fields_id_ptas)):
    #get the field and reduce the dataframe
    field = fields_id_ptas[j]
    params_tmp = params_tmp[params_tmp[field] == ti[j]]
params_tmp = params_tmp.sort_values(by = [field_param])

#data frame to use in copying
df_pt = params_tmp.copy().reset_index(drop = True)
fields_new = list(df_pt[field_param])
df_pt = df_pt[fields_param_years_add_sec].transpose()

dict_conv = dict([x for x in zip(list(df_pt.columns), fields_new)])
df_pt = df_pt.rename(columns = dict_conv).reset_index(drop = True)
df_pt["year"] = [int(x) for x in fields_param_years_add_sec]

#finally, commit changes to params_tmp
params_tmp = params_tmp[params_tmp[field_param].isin(all_params_vary)]
apv = list(params_tmp[field_param])

##  split out max and min
df_pt_max = params_tmp[["sector", field_param, "parameter_constant_q", "max_2050"] + fields_param_years_add_sec].copy().rename(columns = {"max_2050": "range"})
df_pt_min = params_tmp[["sector", field_param, "parameter_constant_q", "min_2050"] + fields_param_years_add_sec].copy().rename(columns = {"min_2050": "range"})

#add fields
df_pt_max["type"] = ["max" for x in range(len(df_pt_max))]
df_pt_min["type"] = ["min" for x in range(len(df_pt_min))]

#build parameters df and initialize futures
params_tmp = pd.concat([df_pt_max, df_pt_min], axis = 0).sort_values(by = [field_param, "type"]).reset_index(drop = True)


In [31]:
if init_fut_q:
    df_future_out = params_tmp[["sector", field_param, "type", "range", "parameter_constant_q"]].copy().reset_index(drop = True)
    df_future_out = df_future_out
    df_future_out["future_id"] = list(range(1, len(df_future_out) + 1))
    #build dict
    dict_fut = dict([[tuple(x[0:2]), int(x[2])] for x in np.array(df_future_out[[field_param, "type", "future_id"]])])
    #reorder
    df_future_out = df_future_out[["future_id", "sector", field_param, "type", "range", "parameter_constant_q"]].rename(columns = {field_param: "parameter", "range": "scale_value", "type": "range_value"})
    init_fut_q = False

In [49]:
[x for x in df_future_out["parameter"] if "trajgroup" in x]
#[x for x in params_tmp["variable_name_lower"] if "trajgroup" in x]

['trajgroup_1-lhs',
 'trajgroup_1-lhs',
 'trajgroup_2-lhs',
 'trajgroup_2-lhs',
 'trajgroup_3-lhs',
 'trajgroup_3-lhs',
 'trajgroup_4-lhs',
 'trajgroup_4-lhs',
 'trajgroup_5-lhs',
 'trajgroup_5-lhs',
 'trajgroup_6-lhs',
 'trajgroup_6-lhs',
 'trajgroup_7-lhs',
 'trajgroup_7-lhs',
 'trajgroup_8-lhs',
 'trajgroup_8-lhs',
 'trajgroup_9-lhs',
 'trajgroup_9-lhs']

In [42]:
#df_future_out[["trajgroup" in x for x in list(df_future_out["parameter"])]]
#array of baseline trajectories
array_traj = np.array(params_tmp[fields_param_years_add_sec])
#rows of this will be the ramp if the parameter is not constant, and 1 otherwise; apply to uncertinaty delta
array_mix = np.array([int(x)*vec_ramp_unc + (1 - int(x))*np.ones(len(fields_param_years_add_sec)) for x in (params_tmp["parameter_constant_q"] == 0)])
#ranges and scalar vector
vec_ranges = np.array(params_tmp["range"])
vec_scale = vec_ranges*np.array(params_tmp["2050"])
vec_unc_delta = vec_scale - np.array(params_tmp["2050"])
#baseline trajectory + the
array_new = array_traj + (array_mix.transpose() * vec_unc_delta).transpose()



In [52]:
np.where(np.array(df_future_out["parameter"]) == "trajgroup_1-lhs")

(array([200, 201]),)

In [58]:
df_future_out[df_future_out["parameter"] == "trajgroup_7-lhs"]

,future_id,sector,parameter,range_value,scale_value,parameter_constant_q
212,213,transport,trajgroup_7-lhs,max,1.0,1
213,214,transport,trajgroup_7-lhs,min,0.0,1


In [ ]:
pt = parameter_table_additional_sectors[parameter_table_additional_sectors["strategy_id"].isin([0])]
pt = pt[pt["time_series_id"].isin([0])]
tg_params = set([sr.clean_trajgroup_names(x)["param"] for x in list(pt["parameter"]) if ("trajgroup" in x) and ("lhs" not in x)])

